## HistogramEqualization

In [ ]:
r"""
 @file HistogramEqualization_hli038.ipynb
 @brief A tity program that implement the basic function of 
         histogram equalization
 
 [AI6121] Computer Vision - Assignment 1 - Histogram Equalization
 
 @author Hantao Li  <hli038@e.ntu.edu.sg>
 @date 01/09/2021
 """

import cv2
import os
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

def CheckPath(path):
    r"""Check if the file in the given path exist."""

    if not (os.path.exists(path)):
        print('Worng Path!')
        exit()


def LoadImage(root_path, image_name):
    r"""Load the iamge from the given path."""

    image_path = root_path + image_name
    CheckPath(image_path)
    image = cv2.imread(filename=image_path)
    return image


def CalHist_RGB(image_b, image_g, image_r):
    r"""Calculate the histogram with the given R, G, B image"""

    hist_b = cv2.calcHist([image_b], [0], None, [256], [0, 256])
    hist_g = cv2.calcHist([image_g], [0], None, [256], [0, 256])
    hist_r = cv2.calcHist([image_r], [0], None, [256], [0, 256])
    return hist_b, hist_g, hist_r


def DrawHist_RGB(image_title, image_b, image_g, image_r):
    r"""Draw the histogram with the given R, G, B image"""

    hist_b, hist_g, hist_r = CalHist_RGB(image_b, image_g, image_r)
    plt.figure()
    plt.title(image_title)
    plt.plot(hist_b, color='b')
    plt.plot(hist_g, color='g')
    plt.plot(hist_r, color='r')
    plt.xlim([0, 256])


def DrawHist_HSV(image_title, image_v):
    r"""Draw the histogram with the single given V/Y image"""

    hist_v = cv2.calcHist([image_v], [0], None, [256], [0, 256])
    plt.figure()
    plt.title(image_title)
    plt.plot(hist_v)
    plt.xlim([0, 256])


def HistogramEqualization_RGB_cv2(image_b, image_g, image_r):
    r"""
    This function is for RGB image.
    Calculate the Histogram Equalization with the cv2 library function 
    For R, G, B channel of the input image. Output is image.
    """

    he_b = cv2.equalizeHist(image_b)
    he_g = cv2.equalizeHist(image_g)
    he_r = cv2.equalizeHist(image_r)
    return he_b, he_g, he_r


def CalHE(image, hist):
    r"""
    Calculate the Histogram Equalization for single input image. 
    Input are the single channel and its Histogram. Output is image.
    """

    #Calculate the probability distribution function of the histogram
    sum_hist = 0
    sum_p = [0] * 256
    for i in range(len(hist)):
        sum_hist = sum_hist + float(hist[i])
        sum_p[i] = sum_hist

    #Use the probability distribution to establish the Lookup Table
    lut = np.zeros(256, dtype=image.dtype)
    for i, ele in enumerate(lut):
        lut[i] = int(255.0 * (sum_p[i] / sum_hist) + 0.5)

    #Use the Lookup Table to calculate the new image after HE
    image_result = lut[image]
    return image_result


def HistogramEqualization_RGB(image_b, image_g, image_r):
    r"""
    This function is for RGB image.
    Calculate the Histogram Equalization for R, G, B channel image. 
    Output are three images, in R, G, B channels.
    """

    hist_b, hist_g, hist_r = CalHist_RGB(image_b, image_g, image_r)
    he_b = CalHE(image_b, hist_b)
    he_g = CalHE(image_g, hist_g)
    he_r = CalHE(image_r, hist_r)
    return he_b, he_g, he_r


def HistogramEqualization_HSV(image_v):
    r"""
    This function is for HSV/YUV image.
    Calculate the Histogram Equalization for V channel image. 
    Output is the new V channel image.
    """

    hist_v = cv2.calcHist([image_v], [0], None, [256], [0, 256])
    he_v = CalHE(image_v, hist_v)
    return he_v

In [ ]:
def HistogramEqualization_AdjustAvarage(image_v):
    r"""
    Re-adjust the output image with HE to the average luminance
    """

    hist_v = cv2.calcHist([image_v], [0], None, [256], [0, 256])
    he_y = CalHE(image_v, hist_v)
    m = np.mean(image_v)
    new_y = he_y.copy()

    if m > 125.5:
        m = 125.5 + 0.5 * (m - 125.5)
        k = 510 - (255**2) / m
        for i, row in enumerate(he_y):
            for j, pixel in enumerate(row):
                if pixel < m:
                    new_y[i][j] = k + (((m - k) / k) * pixel)
    elif m < 125.5:
        m = m + 0.5 * (125.5 - m)
        for i, row in enumerate(he_y):
            for j, pixel in enumerate(row):
                if pixel > m:
                    new_y[i][j] = (((m / (255 - m))**2) * (pixel - m)) + m         
    return new_y

def LR_CalHE(image, hist):
    r"""
    Calculate the Histogram Equalization for single input image. 
    This function use the Left-right method to modify the picture
    Input are the single channel and its Histogram. Output is image.
    """
    
    #Calculate the probability distribution function of the histogram
    m = int(np.mean(image) + 0.5)
    # print(m)
    sum_hist_less = 0
    sum_hist_large = 0
    sum_p_less = [0] * 256
    sum_p_large = [0] * 256
    
    for i in range(len(hist)):
        if i <= m:
            sum_hist_less = sum_hist_less + float(hist[i])
            sum_p_less[i] = sum_hist_less
        else:
            sum_hist_large = sum_hist_large + float(hist[i])
            sum_p_large[i] = sum_hist_large

    #Use the probability distribution to establish the Lookup Table
    lut = np.zeros(256, dtype=image.dtype)
    for i, ele in enumerate(lut):
        if i <= m:
            lut[i] = int((m-1) * (sum_p_less[i] / sum_hist_less) + 0.5)
        else:
            lut[i] = int((255 - m) * (sum_p_large[i] / sum_hist_large) + 0.5) + m
    image_result = lut[image]
    return image_result

def LR_HistogramEqualization_HSV(image_v):
    r"""
    This function is for HSV/YUV image.
    This function use the Left-right method to modify the picture
    Calculate the Histogram Equalization for V/Y channel image. 
    Output is the new V channel image.
    """

    hist_v = cv2.calcHist([image_v], [0], None, [256], [0, 256])
    he_v = LR_CalHE(image_v, hist_v)
    return he_v

In [ ]:
#This part of code is for the 1st step of assignment, which comparing the
#  different HE function result to check the self-write function

#Draw the comparison of image with different HE function.
fig_output_1 = plt.figure(constrained_layout=False, dpi=600)
gs = fig_output_1.add_gridspec(nrows=3, ncols=4, wspace=0.05, hspace=0.2)

a1 = fig_output_1.add_subplot(gs[0, 1:3])
plt.xticks([]), plt.yticks([])
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.title("Original Image", fontsize=5)

a2 = fig_output_1.add_subplot(gs[2, 1:2])
plt.xticks([]), plt.yticks([])
plt.imshow(HEimage_rgb_cv2)
plt.title("HE Image(RGB)-cv2 Function", fontsize=5)

a3 = fig_output_1.add_subplot(gs[1, 1:2])
plt.xticks([]), plt.yticks([])
plt.imshow(HEimage_rgb)
plt.title("HE Image(RGB)", fontsize=5)

a4 = fig_output_1.add_subplot(gs[2, 2:3])
plt.xticks([]), plt.yticks([])
plt.imshow(HEimage_hsv_cv2)
plt.title("HE Image(HSV)-cv2 Function", fontsize=5)

a5 = fig_output_1.add_subplot(gs[1, 2:3])
plt.xticks([]), plt.yticks([])
plt.imshow(HEimage_hsv)
plt.title("HE Image(HSV)", fontsize=5)

plt.savefig("./Output/Sample Comparison.png", dpi=600)

#Draw the Histogram of RGB-channel on original image.
plt.figure(dpi=600)
plt.title("Histogram(RGB)-Original Image")
hist_b, hist_g, hist_r = CalHist_RGB(b, g, r)
plt.plot(hist_b, color='b')
plt.plot(hist_g, color='g')
plt.plot(hist_r, color='r')
plt.xlim([0, 256])
plt.savefig("./Output/Histogram(RGB)-Original Image.png", dpi=600)

#Draw the HE result of RGB with cv2 library function
plt.figure(dpi=600)
plt.title("HE Histogram(RGB)-cv2")
hist_b, hist_g, hist_r = CalHist_RGB(he_b_cv2, he_g_cv2, he_r_cv2)
plt.plot(hist_b, color='b')
plt.plot(hist_g, color='g')
plt.plot(hist_r, color='r')
plt.xlim([0, 256])
plt.savefig("./Output/HE Histogram(RGB)-cv2.png", dpi=600)

#Draw the HE result of RGB with self-write function
plt.figure(dpi=600)
plt.title("HE Histogram(RGB)")
hist_b, hist_g, hist_r = CalHist_RGB(he_b, he_g, he_r)
plt.plot(hist_b, color='b')
plt.plot(hist_g, color='g')
plt.plot(hist_r, color='r')
plt.xlim([0, 256])
plt.savefig("./Output/HE Histogram(RGB).png", dpi=600)

#Draw the Histogram of V-channel on original image.
plt.figure(dpi=600)
plt.title("Histogram(HSV)-Original Image")
hist_v = cv2.calcHist([v], [0], None, [256], [0, 256])
plt.plot(hist_v)
plt.xlim([0, 256])
plt.savefig("./Output/Histogram(HSV)-Original Image.png", dpi=600)

#Draw the HE result of HSV with cv2 library function
plt.figure(dpi=600)
plt.title("HE Histogram(HSV)-cv2")
hist_v = cv2.calcHist([he_v_cv2], [0], None, [256], [0, 256])
plt.plot(hist_v)
plt.xlim([0, 256])
plt.savefig("./Output/HE Histogram(HSV)-cv2.png", dpi=600)

#Draw the HE result of HSV with self-write function
plt.figure(dpi=600)
plt.title("HE Histogram(HSV)")
hist_v = cv2.calcHist([he_v], [0], None, [256], [0, 256])
plt.plot(hist_v)
plt.xlim([0, 256])
plt.savefig("./Output/1/HE Histogram(HSV).png", dpi=600)

In [ ]:
#This part of code is for the 2nd step of assignment, which implementing the
#  HE function on RGB/HSV/YCbCr image for all the 8 images given to
#  choose the best color model(RGB/HSV/YUV) to implement the HE.

#Load the image
root_path = './Images/'
name = os.listdir(root_path)

for image_number, image_name in enumerate(name):

    fig = plt.figure(constrained_layout=False, dpi=600)
    fig.subplots_adjust(wspace=None, hspace=None, right=None, left=None)

    #Draw the original image
    image_current = LoadImage(root_path, image_name)
    fig_ori = fig.add_subplot(1, 4, 1)
    plt.xticks([]), plt.yticks([])
    fig_ori.imshow(cv2.cvtColor(image_current, cv2.COLOR_BGR2RGB))

    #Draw the image after HE in HSV format
    image_hsv = cv2.cvtColor(image_current, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(image_hsv)
    he_hsv = cv2.merge([h, s, HistogramEqualization_HSV(v)])
    HEimage_hsv = cv2.cvtColor(he_hsv, cv2.COLOR_HSV2RGB)
    fig_hsv = fig.add_subplot(1, 4, 2)
    plt.xticks([]), plt.yticks([])
    fig_hsv.imshow(HEimage_hsv)

    #Draw the image after HE in YCrCb format
    image_ycrcb = cv2.cvtColor(image_current, cv2.COLOR_BGR2YCrCb)
    y, cr, cb = cv2.split(image_ycrcb)
    he_ycrcb = cv2.merge([HistogramEqualization_HSV(y), cr, cb])
    HEimage_ycrcb = cv2.cvtColor(he_ycrcb, cv2.COLOR_YCrCb2RGB)
    fig_ycrcb = fig.add_subplot(1, 4, 3)
    plt.xticks([]), plt.yticks([])
    fig_ycrcb.imshow(HEimage_ycrcb)

    #Draw the iamge after HE in RGB format
    b, g, r = cv2.split(image_current)
    he_b, he_g, he_r = HistogramEqualization_RGB(b, g, r)
    he_rgb = cv2.merge([he_b, he_g, he_r])
    HEimage_rgb = cv2.cvtColor(he_rgb, cv2.COLOR_BGR2RGB)
    fig_rgb = fig.add_subplot(1, 4, 4)
    plt.xticks([]), plt.yticks([])
    fig_rgb.imshow(HEimage_rgb)

    #Save the image to the output file
    fig.savefig("./Output/2/HE Comparison_" + str(image_number + 1) + ".png",
                dpi=600)
    plt.close('all')
    print("Finished for Image: " + str(image_name))

In [ ]:
#This part of code is for the 3rd step of assignment, which implementing the
#  HE function on YCbCr image for all the 8 images given.

#Load the image
root_path = './Images/'
name = os.listdir(root_path)

for image_number, image_name in enumerate(name):

    fig = plt.figure(constrained_layout=False, dpi=600)
    fig.subplots_adjust(wspace=None, hspace=None, right=None, left=None)

    #Draw the original image
    image_current = LoadImage(root_path, image_name)
    fig_ori = fig.add_subplot(1, 2, 1)
    plt.xticks([]), plt.yticks([])
    fig_ori.imshow(cv2.cvtColor(image_current, cv2.COLOR_BGR2RGB))

    #Draw the image after HE in YCrCb format
    image_ycrcb = cv2.cvtColor(image_current, cv2.COLOR_BGR2YCrCb)
    y, cr, cb = cv2.split(image_ycrcb)
    he_ycrcb = cv2.merge([HistogramEqualization_HSV(y), cr, cb])
    HEimage_ycrcb = cv2.cvtColor(he_ycrcb, cv2.COLOR_YCrCb2RGB)
    fig_ycrcb = fig.add_subplot(1, 2, 2)
    plt.xticks([]), plt.yticks([])
    fig_ycrcb.imshow(HEimage_ycrcb)

    #Save the image to the output file
    fig.savefig("./Output/3/HE Comparison_" + str(image_number + 1) + ".png",
                dpi=600)
    plt.close('all')
    print("Finished for Image: " + str(image_name))

In [ ]:
# This part of code is for the 4th step of assignment, which implementing the
# Re-approach Average Luminance function.

root_path = './Images/'
name = os.listdir(root_path)

fig = plt.figure(constrained_layout=False, dpi=600)
fig.subplots_adjust(wspace=None, hspace=None, right=None, left=None)

#Draw the original image
image_current = LoadImage(root_path, name[7])
fig_ori = fig.add_subplot(1, 3, 1)
plt.xticks([]), plt.yticks([])
fig_ori.imshow(cv2.cvtColor(image_current, cv2.COLOR_BGR2RGB))

image_ycrcb = cv2.cvtColor(image_current, cv2.COLOR_BGR2YCrCb)
y, cr, cb = cv2.split(image_ycrcb)

#Draw the HE image
he_ycrcb = cv2.merge([HistogramEqualization_HSV(y), cr, cb])
HEimage_ycrcb = cv2.cvtColor(he_ycrcb, cv2.COLOR_YCrCb2RGB)
fig_ycrcb = fig.add_subplot(1, 3, 2)
plt.xticks([]), plt.yticks([])
fig_ycrcb.imshow(HEimage_ycrcb)

#Draw the Re-approach average luminance image
he_ycrcb = cv2.merge([HistogramEqualization_AdjustAvarage(y), cr, cb])
HEimage_ycrcb_new = cv2.cvtColor(he_ycrcb, cv2.COLOR_YCrCb2RGB)
fig_ycrcb_new = fig.add_subplot(1, 3, 3)
plt.xticks([]), plt.yticks([])
fig_ycrcb_new.imshow(HEimage_ycrcb_new)

In [ ]:
#This part of code is for the 5th step of assignment, which implementing the
# different HE function, including HE, Left-right HE, and CLAHE

#Load the image
root_path = './Images/'
name = os.listdir(root_path)
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))

for image_number, image_name in enumerate(name):

    fig = plt.figure(constrained_layout=False, dpi=600)
    fig.subplots_adjust(wspace=None, hspace=None, right=None, left=None)

    #Draw the original image
    image_current = LoadImage(root_path, image_name)
    fig_ori = fig.add_subplot(1, 4, 1)
    plt.xticks([]), plt.yticks([])
    fig_ori.imshow(cv2.cvtColor(image_current, cv2.COLOR_BGR2RGB))

    #Draw the image after HE in YCrCb format
    image_ycrcb = cv2.cvtColor(image_current, cv2.COLOR_BGR2YCrCb)
    y, cr, cb = cv2.split(image_ycrcb)
    he_ycrcb = cv2.merge([HistogramEqualization_HSV(y), cr, cb])
    HEimage_ycrcb = cv2.cvtColor(he_ycrcb, cv2.COLOR_YCrCb2RGB)
    fig_ycrcb = fig.add_subplot(1, 4, 2)
    plt.xticks([]), plt.yticks([])
    fig_ycrcb.imshow(HEimage_ycrcb)
    
    #Draw the image after Left-right HE
    he_ycrcb_B = cv2.merge([LR_HistogramEqualization_HSV(y), cr, cb])
    HEimage_ycrcb_B = cv2.cvtColor(he_ycrcb_B, cv2.COLOR_YCrCb2RGB)
    fig_ycrcb_B = fig.add_subplot(1, 4, 3)
    plt.xticks([]), plt.yticks([])
    fig_ycrcb_B.imshow(HEimage_ycrcb_B)

    #Draw the image after CLAHE
    cl1 = clahe.apply(y)
    res = np.hstack((y, cl1))
    he_ycrcb_C = cv2.merge([cl1, cr, cb])
    HEimage_ycrcb_C = cv2.cvtColor(he_ycrcb_C, cv2.COLOR_YCrCb2RGB)
    fig_ycrcb_C = fig.add_subplot(1, 4, 4)
    plt.xticks([]), plt.yticks([])
    fig_ycrcb_C.imshow(HEimage_ycrcb_C)

    #Save the image to the output file
    fig.savefig("./Output/4/HE Comparison_" + str(image_number + 1) + ".png",
                dpi=600)
    plt.close('all')
    print("Finished for Image: " + str(image_name))